In [ ]:
from typing import Tuple, Literal, TypeVar

from numpy import integer, bool_, floating, ndarray, s_, ones
from plotly.express import line

from degradation_eda.processing_stages import (
    load_data,
    mask_frames,
    estimate_poisson_uncertainty,
    correct_deadtime,
    correct_dark_current,
    correct_frame_time,
    correct_flux_and_transmission,
    correct_self_absorbtion,
    average_frames,
    subtract_background,
    correct_flatfield,
    correct_angular_efficincy,
    correct_solid_angle,
    azimuthally_integrate
)
from degradation_eda.utils.hdf5_filters import load_hdf5_filters
from degradation_eda.utils.visualization import show_frame
from degradation_eda.utils.get_background import get_background

In [ ]:
FILE_PATH = "/dls/i22/data/2021/sm27656-2/i22-588299.nxs"
BACKGROUND_PATH = get_background(FILE_PATH)

In [ ]:
load_hdf5_filters()
NumFrames = TypeVar("NumFrames", bound=int)
NumBackgrounds = TypeVar("NumBackgrounds", bound=int)
frames: ndarray[Tuple[NumFrames, Literal[1679], Literal[1475]], integer] = load_data(FILE_PATH, "entry1/instrument/detector/data", s_[740:750, 0, :, :]).squeeze()
background: ndarray[Tuple[NumBackgrounds, Literal[1679], Literal[1475]], integer] = load_data(BACKGROUND_PATH, "entry1/instrument/detector/data", s_[0, :10, :, :]).squeeze()

In [ ]:
mask: ndarray[Tuple[Literal[1679], Literal[1475]], bool_] = load_data(FILE_PATH, "entry1/instrument/detector/pixel_mask")
frames = mask_frames(frames, mask)
background = mask_frames(background, mask)

In [ ]:
uncert_frames = estimate_poisson_uncertainty(frames)
uncert_background = estimate_poisson_uncertainty(background)

In [ ]:
count_times: ndarray[Tuple[Literal[10]], floating] = load_data(FILE_PATH, "entry1/instrument/detector/count_time", s_[740:750, 0]).squeeze()
dead_times: ndarray[Tuple[Literal[10]], floating] = load_data(FILE_PATH, "entry1/instrument/detector/wait_time", s_[740:750, 0]).squeeze()

uncert_frames = correct_deadtime(uncert_frames, count_times, dead_times)
uncert_background = correct_deadtime(uncert_background, count_times[0], dead_times[0])

In [ ]:
uncert_frames = correct_dark_current(uncert_frames, 0.0)
uncert_background = correct_dark_current(uncert_background, 0.0)

In [ ]:
uncert_frames = correct_frame_time(uncert_frames, count_times, dead_times)
uncert_background = correct_frame_time(uncert_background, count_times[0], dead_times[0])

In [ ]:
uncert_frames = correct_flux_and_transmission(uncert_frames)
uncert_background = correct_flux_and_transmission(uncert_background)

In [ ]:
beam_center_pixels = (
    load_data(FILE_PATH, "entry1/instrument/detector/beam_center_y"),
    load_data(FILE_PATH, "entry1/instrument/detector/beam_center_x")
)
pixel_sizes = (
    load_data(FILE_PATH, "entry1/instrument/detector/y_pixel_size"),
    load_data(FILE_PATH, "entry1/instrument/detector/x_pixel_size")
)
distance = load_data(FILE_PATH, "entry1/instrument/detector/distance")
transmissibility = 0.1

uncert_frames = correct_self_absorbtion(
    uncert_frames,
    beam_center_pixels,
    pixel_sizes,
    distance,
    transmissibility
    )
uncert_background = correct_self_absorbtion(
    uncert_background,
    beam_center_pixels,
    pixel_sizes,
    distance,
    transmissibility
)

In [ ]:
azimuthally_intensities, azimuths = azimuthally_integrate(
    uncert_frames[0]["nominal"],
    beam_center_pixels,
    pixel_sizes,
    distance
)
line(x=azimuths, y=azimuthally_intensities, log_x=True, log_y=True)

In [ ]:
uncert_background = average_frames(uncert_background)

In [ ]:
diff_frames = subtract_background(uncert_frames, uncert_background)

In [ ]:
diff_frames = correct_flatfield(diff_frames, ones(diff_frames.shape[1:], dtype=floating))

In [ ]:
diff_frames = correct_angular_efficincy(
    diff_frames,
    beam_center_pixels,
    pixel_sizes,
    distance,
    0.01,
    450E-6
)

In [ ]:
diff_frames = correct_solid_angle(
    diff_frames,
    beam_center_pixels,
    pixel_sizes,
    distance
)

In [ ]:
show_frame(diff_frames[0]["nominal"].filled(0.0))